# Self-Driving Car Engineer Nanodegree


## Project: **Finding Lane Lines on the Road** 
***
In this project, you will use the tools you learned about in the lesson to identify lane lines on the road.  You can develop your pipeline on a series of individual images, and later apply the result to a video stream (really just a series of images). Check out the video clip "raw-lines-example.mp4" (also contained in this repository) to see what the output should look like after using the helper functions below. 

Once you have a result that looks roughly like "raw-lines-example.mp4", you'll need to get creative and try to average and/or extrapolate the line segments you've detected to map out the full extent of the lane lines.  You can see an example of the result you're going for in the video "P1_example.mp4".  Ultimately, you would like to draw just one line for the left side of the lane, and one for the right.

In addition to implementing code, there is a brief writeup to complete. The writeup should be completed in a separate file, which can be either a markdown file or a pdf document. There is a [write up template](https://github.com/udacity/CarND-LaneLines-P1/blob/master/writeup_template.md) that can be used to guide the writing process. Completing both the code in the Ipython notebook and the writeup template will cover all of the [rubric points](https://review.udacity.com/#!/rubrics/322/view) for this project.

---
Let's have a look at our first image called 'test_images/solidWhiteRight.jpg'.  Run the 2 cells below (hit Shift-Enter or the "play" button above) to display the image.

**Note: If, at any point, you encounter frozen display windows or other confounding issues, you can always start again with a clean slate by going to the "Kernel" menu above and selecting "Restart & Clear Output".**

---

**The tools you have are color selection, region of interest selection, grayscaling, Gaussian smoothing, Canny Edge Detection and Hough Tranform line detection.  You are also free to explore and try other techniques that were not presented in the lesson.  Your goal is piece together a pipeline to detect the line segments in the image, then average/extrapolate them and draw them onto the image for display (as below).  Once you have a working pipeline, try it out on the video stream below.**

---

<figure>
 <img src="examples/line-segments-example.jpg" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Your output should look something like this (above) after detecting line segments using the helper functions below </p> 
 </figcaption>
</figure>
 <p></p> 
<figure>
 <img src="examples/laneLines_thirdPass.jpg" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Your goal is to connect/average/extrapolate line segments to get output like this</p> 
 </figcaption>
</figure>

**Run the cell below to import some packages.  If you get an `import error` for a package you've already installed, try changing your kernel (select the Kernel menu above --> Change Kernel).  Still have problems?  Try relaunching Jupyter Notebook from the terminal prompt.  Also, consult the forums for more troubleshooting tips.**  

## Import Packages

In [ ]:
#importing some useful packages
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
%matplotlib inline

## Read in an Image

In [ ]:
#reading in an image
image = mpimg.imread('test_images/solidWhiteCurve.jpg')
ysize = image.shape[0] # 540
xsize = image.shape[1] # 960
#printing out some stats and plotting
print('This image is:', type(image), 'with dimensions:', image.shape)
plt.imshow(image)  # if you wanted to show a single color channel image called 'gray', for example, call as plt.imshow(gray, cmap='gray')

## Ideas for Lane Detection Pipeline

**Some OpenCV functions (beyond those introduced in the lesson) that might be useful for this project are:**

`cv2.inRange()` for color selection  
`cv2.fillPoly()` for regions selection  
`cv2.line()` to draw lines on an image given endpoints  
`cv2.addWeighted()` to coadd / overlay two images
`cv2.cvtColor()` to grayscale or change color
`cv2.imwrite()` to output images to file  
`cv2.bitwise_and()` to apply a mask to an image

**Check out the OpenCV documentation to learn about these and discover even more awesome functionality!**

## My Thought Process 

***
In the project report, I will present two methods that I developed. 

### Method 1 

Image Pre-processing: grayscale -> canny edge detection -> excessive gaussian blur to provide the single-line lane marks.     

Methodology: using the maximum brightness detection in each rows in the defined regions of interest, which are left and right. After finding the points in (row, column) that indicating the lanes, I used linear regression to interpolate/extrapolate the lanes. I found that method 1 is more stable than the Method 2. 

### Method 2  
Image Pre-processing: grayscale -> canny edge detection -> gentle gaussian blur to outline the lanes. 

Methodology: Use hough transformation to find all the lines in the defined regions of interest, which are left and right. To filter the slope of lines in a reasonable range, and interpolate/extrapolate selected lines to the top and bottom highlights. Average the top and bottom in x-position and link them as two single lines. 

Test sample images: this method is much more sensitive to the parameters I used.  s
Test sample videos: The segmented lane is shaky.   


## Helper Functions

Below are some helper functions to help get you started. They should look familiar from the lesson!

In [ ]:
import math
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
%matplotlib inline

def grayscale(img):
    return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Or use BGR2GRAY if you read an image with cv2.imread()
    # return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
def canny(img, low_threshold, high_threshold):
    return cv2.Canny(img, low_threshold, high_threshold)

def gaussian_blur(img, kernel_size):
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def region_of_interest(img, vertices):
    #defining a blank mask to start with
    mask = np.zeros_like(img)    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image

def draw_lines(img, lines, color=[255, 0, 0], thickness=2):
    for line in lines:
        for x1,y1,x2,y2 in line:
            cv2.line(img, (x1, y1), (x2, y2), color, thickness)

def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    draw_lines(line_img, lines)
    return line_img

# Python 3 has support for cool math symbols.

def weighted_img(img, initial_img, α=0.8, β=1., γ=0.):
    return cv2.addWeighted(initial_img, α, img, β, γ) 

def plot(img, cmap='gray'):
    plt.imshow(img, cmap=cmap)
    plt.show()

## Test Images

Build your pipeline to work on the images in the directory "test_images"  
**You should make sure your pipeline works well on these images before you try the videos.**

In [ ]:
import os
test_image_list= os.listdir("test_images/")

for img_name in test_image_list:
    print (img_name[:-4])
    test_img = mpimg.imread("test_images/" + img_name[:-4]+'.jpg')    

## Build a Lane Finding Pipeline - Method 1



Build the pipeline and run your solution on all test_images. Make copies into the `test_images_output` directory, and you can use the images in your writeup report.

Try tuning the various parameters, especially the low and high Canny thresholds as well as the Hough lines parameters.

In [ ]:
# TODO: Build your pipeline that will draw lane lines on the test_images
# then save them to the test_images_output directory.

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
%matplotlib inline

import os
test_img_list= os.listdir("test_images/")

for img_name in test_img_list:
    print (img_name)
    test_img = mpimg.imread("test_images/" + img_name)

# Grab the x and y size and make a copy of the image
    ysize = test_img.shape[0] # 540
    xsize = test_img.shape[1] # 960
    
# Applies the Grayscale transform  
    gray_img=grayscale(test_img)
    #print("Convert to grayscale:")
    #plot(gray_img)
    #plt.imsave("test_images_output/" + img_name[:-4] + "_gray.jpg", gray_img, cmap='gray')
    
# Applies the Canny transform
    low_threshold=200
    high_threshold=250
    #print("Apply Canny transform")
    canny_img=canny(gray_img, low_threshold, high_threshold)
    #plot(canny_img, cmap='gray')
    #plt.imsave("test_images_output/" + img_name[:-4] + "_canny.jpg", canny_img, cmap='gray')
    
# Apply a Gaussian Noise kernel
    #print("Apply Gaussian Noise")
    blurred_img=gaussian_blur(canny_img, 51)
    #plot(blurred_img)
    #plt.imsave("test_images_output/" + img_name[:-4] + "_blurred.jpg", blurred_img, cmap='gray')
    
# Apply an image mask = left and right 
    vertices_right= np.array([[[xsize*0.5,ysize*0.6],[xsize*0.55,ysize*0.6],[xsize*0.95,ysize*0.9],[xsize*0.5,ysize*0.9]]], dtype=np.int32)
    vertices_left= np.array([[[xsize*0.1,ysize*0.9],[xsize*0.45,ysize*0.6],[xsize*0.5,ysize*0.6],[xsize*0.5,ysize*0.9]]], dtype=np.int32)
    region_right=region_of_interest(blurred_img,vertices_right)
    region_left=region_of_interest(blurred_img,vertices_left)

# Find the max brightness after left_mask image
    data_left=np.argmax(region_left,axis=1)
    x_left=np.arange(len(data_left));
    data_left_extracted= data_left[data_left > 10]
    x_left_extracted=x_left[data_left > 10]
    fitplot_left=np.poly1d(np.polyfit(x_left_extracted,data_left_extracted, 1))

# Find the max brightness after right_mask image
    data_right=np.argmax(region_right,axis=1)
    x_right=np.arange(len(data_right));
    data_right_extracted= data_right[data_right > 10]
    x_right_extracted=x_right[data_right > 10]
    fitplot_right=np.poly1d(np.polyfit(x_right_extracted,data_right_extracted, 1))

# extroploate the points of interest
    line_image = np.copy(test_img)*0 # creating a blank to draw lines on
    lines=[[[int(fitplot_left(ysize)),int(ysize),int(fitplot_left(ysize*0.6)),int(ysize*0.6)],[int(fitplot_right(ysize)),int(ysize),int(fitplot_right(ysize*0.6)),int(ysize*0.6)]]]
    draw_lines(line_image, lines, color=[255, 0, 0], thickness=10)
    lines_edges = weighted_img(line_image, test_img, 0.9, 1, 0) 
    #plt.imshow(lines_edges, cmap='gray')
    #plt.show()
    plot(lines_edges, cmap='gray')
    #plt.imsave("test_images_output/" + img_name[:-4] + "_final.jpg", lines_edges, cmap='gray')


## Build a Lane Finding Pipeline - Method 2

In [ ]:
# TODO: Build your pipeline that will draw lane lines on the test_images
# then save them to the test_images_output directory.

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
%matplotlib inline

import os
test_img_list= os.listdir("test_images/")

for img_name in test_img_list:
    print (img_name)
    test_img = mpimg.imread("test_images/" + img_name)
    test_image = mpimg.imread('test_images/solidWhiteCurve.jpg')

# Grab the x and y size and make a copy of the image
    ysize = test_img.shape[0] # 540
    xsize = test_img.shape[1] # 960

# Applies the Grayscale transform  
    gray_img=grayscale(test_img)
    #print("Convert to grayscale:")
    #plot(gray_img)
    #plt.imsave("test_images_output/" + img_name[:-4] + "_gray_2.jpg", gray_img, cmap='gray')
    
# Applies the Canny transform
    low_threshold=200
    high_threshold=240
    #print("Apply Canny transform")
    canny_img=canny(gray_img, low_threshold, high_threshold)
    #plot(canny_img, cmap='gray')
    #plt.imsave("test_images_output/" + img_name[:-4] + "_canny.jpg_2", canny_img, cmap='gray')

# Apply a Gaussian Noise kernel
    print("Apply Gaussian Blur")
    blurred_img=gaussian_blur(canny_img, 11)
    #plot(blurred_img)
    #plt.imsave("test_images_output/" + img_name[:-4] + "_blurred.jpg_2", blurred_img, cmap='gray')
    
# Applies an image mask.
    vertices_right= np.array([[[xsize*0.5,ysize*0.55],[xsize*0.55,ysize*0.55],[xsize*0.9,ysize],[xsize*0.5,ysize]]], dtype=np.int32)
    vertices_left= np.array([[[xsize*0.1,ysize],[xsize*0.45,ysize*0.55],[xsize*0.5,ysize*0.55],[xsize*0.5,ysize]]], dtype=np.int32)
    region_right=region_of_interest(blurred_img,vertices_right)
    region_left=region_of_interest(blurred_img,vertices_left)

# Apply Hough Lines
    rho = 1 # distance resolution in pixels of the Hough grid
    theta = np.pi/180 # angular resolution in radians of the Hough grid
    threshold = 5     # minimum number of votes (intersections in Hough grid cell)
    min_line_length = 100 #minimum number of pixels making up a line
    max_line_gap = 100    # maximum gap in pixels between connectable line segments
    line_image = np.copy(test_img)*0 # creating a blank to draw lines on

# Average the position of each of the lines and extrapolate to the top and bottom of the lane.  
    lines_right = cv2.HoughLinesP(region_right, rho, theta, threshold, np.array([]),min_line_length, max_line_gap)
    lines_left = cv2.HoughLinesP(region_left, rho, theta, threshold, np.array([]),min_line_length, max_line_gap)
    line_correct_right=[]
    line_correct_left=[]
    slope_right=[]
    slope_left=[]
    x1_right=[]
    x2_right=[]
    x1_left=[]
    x2_left=[]

    for line in lines_right:
        for x1,y1,x2,y2 in line:
            #slope= (y1-y2)/(x1-x2)
            [slope,b] = np.polyfit([y1, y2],[x1, x2], 1)
            if slope < 2 and slope > 1: 
                #line_correct_right.append(line) 
                slope_right.append(slope)            
                f=np.poly1d([slope,b])
                x1_right.append(f(ysize*0.6))
                x2_right.append(f(ysize))
    for line in lines_left:
        for x1,y1,x2,y2 in line:
            #slope= (y1-y2)/(x1-x2)
            [slope,b] = np.polyfit([y1, y2],[x1, x2],1)
            if slope < -1 and slope > -2: 
                #line_correct_left.append(line)  
                slope_left.append(slope)
                f=np.poly1d([slope,b])
                x1_left.append(f(ysize*0.6))
                x2_left.append(f(ysize))
            
    line_correct=[[[int(np.mean(x1_right)),int(ysize*0.6),int(np.mean(x2_right)),int(ysize)],[int(np.mean(x1_left)),int(ysize*0.6),int(np.mean(x2_left)),int(ysize)]]]
    draw_plot=draw_lines(line_image, line_correct, color=[255, 0, 0], thickness=10)
    lines_edges = cv2.addWeighted(test_img, 0.8, line_image, 1, 0) 
    
    #print(line_correct)
    #print(slope_right)
    #print(slope_left)
    plot(lines_edges, cmap='gray')
    #plt.imsave("test_images_output/" + img_name[:-4] + "_final_2.jpg", lines_edges, cmap='gray')

## Test on Videos

You know what's cooler than drawing lanes over images? Drawing lanes over video!

We can test our solution on two provided videos:

`solidWhiteRight.mp4`

`solidYellowLeft.mp4`

**Note: if you get an import error when you run the next cell, try changing your kernel (select the Kernel menu above --> Change Kernel). Still have problems? Try relaunching Jupyter Notebook from the terminal prompt. Also, consult the forums for more troubleshooting tips.**

**If you get an error that looks like this:**
```
NeedDownloadError: Need ffmpeg exe. 
You can download it by calling: 
imageio.plugins.ffmpeg.download()
```
**Follow the instructions in the error message and check out [this forum post](https://discussions.udacity.com/t/project-error-of-test-on-videos/274082) for more troubleshooting tips across operating systems.**

In [ ]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

## Pipeline 1 based on Method 1

In [ ]:
# Pipeline 1 

def process_image(image):
    # NOTE: The output you return should be a color image (3 channel) for processing video below
    # TODO: put your pipeline here,
    # you should return the final output (image where lines are drawn on lanes)
    
    t = 60
    low_threshold=200 #100
    high_threshold=250
    
    # make a copy
    test_img = np.copy(image)
    ysize = test_img.shape[0] # 540
    xsize = test_img.shape[1] # 960
    # PIPELINE
    # 0 convert to grayscale
    gray_img=grayscale(test_img)    
    # 1 apply the Canny transform
    canny_img=canny(gray_img, low_threshold, high_threshold)
    # 2 apply blur
    blurred_img = gaussian_blur(canny_img, 51)
    # 3 apply ROI mask
    # Apply an image mask = left and right 
    vertices_right= np.array([[[xsize*0.5,ysize*0.65],[xsize*0.55,ysize*0.65],[xsize*0.95,ysize*0.85],[xsize*0.5,ysize*0.85]]], dtype=np.int32)
    vertices_left= np.array([[[xsize*0.1,ysize*0.85],[xsize*0.45,ysize*0.65],[xsize*0.5,ysize*0.65],[xsize*0.5,ysize*0.85]]], dtype=np.int32)
    region_right=region_of_interest(blurred_img,vertices_right)
    region_left=region_of_interest(blurred_img,vertices_left)

    # Find the max brightness after left_mask image
    data_left=np.argmax(region_left,axis=1)
    x_left=np.arange(len(data_left));
    data_left_extracted= data_left[data_left > 10]
    x_left_extracted=x_left[data_left > 10]
    fitplot_left=np.poly1d(np.polyfit(x_left_extracted,data_left_extracted, 1))

    # Find the max brightness after right_mask image
    data_right=np.argmax(region_right,axis=1)
    x_right=np.arange(len(data_right));
    data_right_extracted= data_right[data_right > 10]
    x_right_extracted=x_right[data_right > 10]
    fitplot_right=np.poly1d(np.polyfit(x_right_extracted,data_right_extracted, 1))
    
    # 4 get lines image 
    # extroploate the points of interest
    line_image = np.copy(test_img)*0 # creating a blank to draw lines on
    lines=[[[int(fitplot_left(ysize)),int(ysize),int(fitplot_left(ysize*0.65)),int(ysize*0.65)],[int(fitplot_right(ysize)),int(ysize),int(fitplot_right(ysize*0.65)),int(ysize*0.65)]]]
    draw_lines(line_image, lines, color=[255, 0, 0], thickness=10)
    
    # 5 superimpose result on top of original image
    final_img = weighted_img(line_image, test_img, 0.9, 1, 0)
   
    #final_img = weighted_img(lines_edges, test_img)
    return final_img

## Pipeline 2 based on Method 2

In [ ]:
# Pipeline 2

def process_image(image):
    # NOTE: The output you return should be a color image (3 channel) for processing video below
    # TODO: put your pipeline here,
    # you should return the final output (image where lines are drawn on lanes)
    
    t = 60
    low_threshold=200
    high_threshold=240
    
    # make a copy
    test_img = np.copy(image)
    ysize = test_img.shape[0] # 540
    xsize = test_img.shape[1] # 960
    # PIPELINE
    # 0 convert to grayscale
    gray_img=grayscale(test_img)    
    # 1 apply the Canny transform
    canny_img=canny(gray_img, low_threshold, high_threshold)
    # 2 apply blur
    blurred_img = gaussian_blur(canny_img, 11)
    # 3 apply ROI mask
    vertices_right= np.array([[[xsize*0.5,ysize*0.55],[xsize*0.55,ysize*0.55],[xsize*0.9,ysize],[xsize*0.5,ysize]]], dtype=np.int32)
    vertices_left= np.array([[[xsize*0.1,ysize],[xsize*0.45,ysize*0.55],[xsize*0.5,ysize*0.55],[xsize*0.5,ysize]]], dtype=np.int32)
    region_right=region_of_interest(blurred_img,vertices_right)
    region_left=region_of_interest(blurred_img,vertices_left)

    # 4 apply Hough Lines
    rho = 1 # distance resolution in pixels of the Hough grid
    theta = np.pi/180 # angular resolution in radians of the Hough grid
    threshold = 5     # minimum number of votes (intersections in Hough grid cell)
    min_line_length = 100 #minimum number of pixels making up a line
    max_line_gap = 100    # maximum gap in pixels between connectable line segments
    
    # 5 Average the position of each of the lines and extrapolate to the top and bottom of the lane.  
    lines_right = cv2.HoughLinesP(region_right, rho, theta, threshold, np.array([]),min_line_length, max_line_gap)
    lines_left = cv2.HoughLinesP(region_left, rho, theta, threshold, np.array([]),min_line_length, max_line_gap)
    line_correct_right=[]
    line_correct_left=[]
    slope_right=[]
    slope_left=[]
    x1_right=[]
    x2_right=[]
    x1_left=[]
    x2_left=[]

    for line in lines_right:
        for x1,y1,x2,y2 in line:
            #slope= (y1-y2)/(x1-x2)
            [slope,b] = np.polyfit([y1, y2],[x1, x2], 1)
            if slope < 2 and slope > 1: 
                #line_correct_right.append(line) 
                slope_right.append(slope)            
                f=np.poly1d([slope,b])
                x1_right.append(f(ysize*0.6))
                x2_right.append(f(ysize))
    for line in lines_left:
        for x1,y1,x2,y2 in line:
            #slope= (y1-y2)/(x1-x2)
            [slope,b] = np.polyfit([y1, y2],[x1, x2],1)
            if slope < -1 and slope > -2: 
                #line_correct_left.append(line)  
                slope_left.append(slope)
                f=np.poly1d([slope,b])
                x1_left.append(f(ysize*0.6))
                x2_left.append(f(ysize))
             
    # 6 get lines image 
    line_image = np.copy(image)*0 # creating a blank to draw lines on
    line_correct=[[[int(np.mean(x1_right)),int(ysize*0.6),int(np.mean(x2_right)),int(ysize)],[int(np.mean(x1_left)),int(ysize*0.6),int(np.mean(x2_left)),int(ysize)]]]
    draw_plot=draw_lines(line_image, line_correct, color=[255, 0, 0], thickness=10)
                
    # 7 superimpose result on top of original image
    final_img = weighted_img(line_image, test_img, 0.9, 1, 0)
   
    #final_img = weighted_img(lines_edges, test_img)
    return final_img

Let's try the one with the solid white lane on the right first ...

In [ ]:
image = mpimg.imread('test_images/solidWhiteCurve.jpg')
abc=process_image(image)
plt.imshow(abc)

In [ ]:
white_output = 'test_videos_output/solidWhiteRight.mp4'
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
##clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4").subclip(0,5)
clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4")
white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)

Play the video inline, or if you prefer find the video in your filesystem (should be in the same directory) and play it in your video player of choice.

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))

## Improve the draw_lines() function

**At this point, if you were successful with making the pipeline and tuning parameters, you probably have the Hough line segments drawn onto the road, but what about identifying the full extent of the lane and marking it clearly as in the example video (P1_example.mp4)?  Think about defining a line to run the full length of the visible lane based on the line segments you identified with the Hough Transform. As mentioned previously, try to average and/or extrapolate the line segments you've detected to map out the full extent of the lane lines. You can see an example of the result you're going for in the video "P1_example.mp4".**

**Go back and modify your draw_lines function accordingly and try re-running your pipeline. The new output should draw a single, solid line over the left lane line and a single, solid line over the right lane line. The lines should start from the bottom of the image and extend out to the top of the region of interest.**

Now for the one with the solid yellow lane on the left. This one's more tricky!

In [ ]:
yellow_output = 'test_videos_output/solidYellowLeft.mp4'
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
##clip2 = VideoFileClip('test_videos/solidYellowLeft.mp4').subclip(0,5)
clip2 = VideoFileClip('test_videos/solidYellowLeft.mp4')
yellow_clip = clip2.fl_image(process_image)
%time yellow_clip.write_videofile(yellow_output, audio=False)

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(yellow_output))

## Optional Challenge

Try your lane finding pipeline on the video below.  Does it still work?  Can you figure out a way to make it more robust?  If you're up for the challenge, modify your pipeline so it works with this video and submit it along with the rest of your project!

In [ ]:
challenge_output = 'test_videos_output/challenge.mp4'
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
##clip3 = VideoFileClip('test_videos/challenge.mp4').subclip(0,5)
clip3 = VideoFileClip('test_videos/challenge.mp4')
challenge_clip = clip3.fl_image(process_image)
%time challenge_clip.write_videofile(challenge_output, audio=False)

## Reflection

Please see all 3 videos in the folder incluing the challenge. 

Method 1 is more robust than Method 2, as you can see that most of the time, method 1 can find the lanes accurately in both solidWhiteRight.mp4 and solidYellowLeft.mp4. 

To manage this challenge.mp4 is the best practice to validate the robustness of the pipeline. There are a few improvement I made to make finding the lanes in the challenge video based on Method 1. Still it is not preferct. 

Here are some ideas of improvement. 

(1) The bottom of images was covered by the body of vehicle that certainly disturb the pipeline process. To avoid that, simply to make the region of interest smaller and avoid that area of image.

(2) I also noticed that the pipeline failed when car was driven under tree shade, so the next step is to sharpen the lanes in such image. 

(3) When the road condition changes, from old gray road to new black asphalt road, there are the horizontal lines/features to interfere the original algorithm. Therefore, if I combine method 1 and method 2, then I could have avoid this. 

(4) Method 1 can be exptended to curved lane by making the trajectory.  

Overall, this is a fun project, and I sharpen my basic python skills in a short time.
